In [39]:
import pandas as pd
from textblob import TextBlob
import csv 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
#import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers
import re
import numpy as np

In [158]:
df = pd.read_csv('Resources/airlines_reviews.csv')

In [3]:
#some work on the columns #1

def column_work(df):
    
    #create new db with only the needed columns
    
    df_new = df[['Title', 'Airline', 'Reviews',
       'Type of Traveller', 'Month Flown', 'Route', 'Class', 'Seat Comfort',
       'Staff Service', 'Food & Beverages', 'Inflight Entertainment',
       'Value For Money', 'Overall Rating', 'Recommended']]
    
    #split the month flown column into two and make the month into a number
        # first map the month names to their numbers

    num_mon = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
                'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}

    df_new['Year Flown'] = df_new['Month Flown'].str.split().str[1]
    df_new['Month Flown'] = df_new['Month Flown'].str.split().str[0].map(num_mon)
    #df_new.loc[:,'Year Flown'] = df_new[:'Month Flown'].str.split().str[1]
    #df_new.loc[:, 'Month Flown'] = df_new.loc[:, 'Month Flown'].str.split().str[0].map(num_mon)

    df_new.head()

    return df_new

In [159]:
#1
df_c = column_work(df) 

/var/folders/gw/49dbrphx3bd0c4cm_7kpyb_m0000gn/T/ipykernel_4963/4156166323.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Year Flown'] = df_new['Month Flown'].str.split().str[1]
/var/folders/gw/49dbrphx3bd0c4cm_7kpyb_m0000gn/T/ipykernel_4963/4156166323.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Month Flown'] = df_new['Month Flown'].str.split().str[0].map(num_mon)


In [184]:
# split the route column #2
def split_to_via(df):
    # Split the text based on separators (to, via)
    split_values = df['Route'].str.split(r'(?<=\bto\b)|(?<=\bvia\b)', expand=True)
    
    # remove to and via from the split columns
    split_values = split_values.apply(lambda x: x.str.replace(r'\bto\b|\bvia\b', '', regex=True))
    
    # Create new columns and load in the split values
    df['Origin'] = split_values[0].str.strip()
    df['Destination'] = split_values[1].str.strip()
    
    # Check if the 'Via' column exists and handle multiple cities
    if 2 in split_values.columns:
        df['Via'] = split_values[2].str.replace('/', '').str.split().str[0].str.strip()
    else:
        df['Via'] = ''
    
    return df

In [175]:
#2
df_d = split_to_via_2(df_c)

In [5]:
## text blobs #3
def blob_function(df):

    # extract the comments and ratings into a list

    titles = df['Title'].to_list()
    comments = df['Reviews'].to_list()

    # create a blank list to hold the blob

    polarities = []
    subjectivities = []

    # Analyze the sentiment of each comment 

    for comment in comments:
      blob = TextBlob(comment)
      polarity = blob.sentiment.polarity
      subjectivity = blob.sentiment.subjectivity
      polarities.append(polarity)
      subjectivities.append(subjectivity)
    
     
    # add values to new df columns

    df['Comment Polarity'] = pd.Series(polarities)
    df['Comment Subjectivity'] = pd.Series(subjectivities)

    # erase lists to hold the blob

    polarities = []
    subjectivities = []    

    # Analyze the sentiment of each title

    for title in titles:
      blob = TextBlob(title)
      polarity = blob.sentiment.polarity
      subjectivity = blob.sentiment.subjectivity
      polarities.append(polarity)
      subjectivities.append(subjectivity)

     # add values to new df columns

    df['Title Polarity'] = pd.Series(polarities)
    df['Title Subjectivity'] = pd.Series(subjectivities)
    
    return df

In [ ]:
#3
df_e = split_to_via(df_d)

In [6]:
# begin the one hot encoding #4

def one_hot_e (df):
    # get dummies 
    df = pd.get_dummies(df, columns = ['Class', 'Type of Traveller'])
    return df

In [ ]:
df_f = column_work(df_e) 


In [7]:
# more column work #5

def col_work_dos(df):
    #drop some columns
    
    df_renew = df[[ 'Airline', 'Month Flown', 'Year Flown', 'Seat Comfort',
       'Staff Service', 'Food & Beverages', 'Inflight Entertainment',
       'Value For Money', 'Overall Rating', 'Recommended', 
       'Origin', 'Destination', 'Via', 'Comment Polarity',
       'Comment Subjectivity', 'Title Polarity', 'Title Subjectivity',
       'Class_Business Class', 'Class_Economy Class', 'Class_First Class',
       'Class_Premium Economy', 'Type of Traveller_Business',
       'Type of Traveller_Couple Leisure', 'Type of Traveller_Family Leisure',
       'Type of Traveller_Solo Leisure']]
    
    
    # reformat recommended column values
    df_renew ['Recommended'] = df_renew['Recommended'].replace({'no':0, 'yes':1})

    return df_renew

In [4]:
df_g = column_work(df_f) 


NameError: name 'column_work' is not defined

In [17]:
df_data.head(3)

,Airline,Month Flown,Year Flown,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended,...,Title Polarity,Title Subjectivity,Class_Business Class,Class_Economy Class,Class_First Class,Class_Premium Economy,Type of Traveller_Business,Type of Traveller_Couple Leisure,Type of Traveller_Family Leisure,Type of Traveller_Solo Leisure
0,Singapore Airlines,12,2023,4,4,4,4,4,9,1,...,0.6,0.9,True,False,False,False,False,False,False,True
1,Singapore Airlines,2,2024,5,3,4,4,1,3,0,...,-1.0,1.0,False,True,False,False,False,False,False,True
2,Singapore Airlines,2,2024,1,5,2,1,5,10,1,...,0.0,0.0,False,True,False,False,False,False,True,False


In [13]:
hot_df.columns

Index(['Title', 'Airline', 'Reviews', 'Month Flown', 'Route', 'Seat Comfort',
       'Staff Service', 'Food & Beverages', 'Inflight Entertainment',
       'Value For Money', 'Overall Rating', 'Recommended', 'Year Flown',
       'Origin', 'Destination', 'Via', 'Comment Polarity',
       'Comment Subjectivity', 'Title Polarity', 'Title Subjectivity',
       'Class_Business Class', 'Class_Economy Class', 'Class_First Class',
       'Class_Premium Economy', 'Type of Traveller_Business',
       'Type of Traveller_Couple Leisure', 'Type of Traveller_Family Leisure',
       'Type of Traveller_Solo Leisure'],
      dtype='object')

In [14]:
df_blob.columns

Index(['Title', 'Airline', 'Reviews', 'Type of Traveller', 'Month Flown',
       'Route', 'Class', 'Seat Comfort', 'Staff Service', 'Food & Beverages',
       'Inflight Entertainment', 'Value For Money', 'Overall Rating',
       'Recommended', 'Year Flown', 'Origin', 'Destination', 'Via',
       'Comment Polarity', 'Comment Subjectivity', 'Title Polarity',
       'Title Subjectivity'],
      dtype='object')

In [15]:
def machine_fun(df):
    #begin the machine

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state = 43)
    

In [16]:
rate = df['Overall Rating']
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
scatter = plt.scatter(polarities, subjectivities, c=rate, cmap='viridis', alpha=0.5)
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.title('Polarity vs. Subjectivity')
plt.show()

NameError: name 'polarities' is not defined

<Figure size 800x600 with 0 Axes>

In [ ]:
df_blob["Class"].unique()

In [22]:
via_=df_blob['Via'].unique()

In [26]:
via_=df_blob['Via'].unique()
for a in via_: print(a)

None
Singapore
Singapore / Frankfurt
Rome
Singaphore
Narita
Incheon
Singapore/Sydney
Munich
Tokyo, Singapore
Frankfurt
Sydney
Tokyo
Melbourne
Johannesburg
Melbourne / Singapore
Moscow
Milan / Singapore
Singapore and Seoul
Melbourne, Singapore
Canberra
Hong Kong
SIN
HKG
MAN
NRT / SIN
PER
SiN
JNB
Doha
Phuket
Dohs
dona
Montreal & Doha
Doha / Perth
Doha, Qatar
Doha & Hong Kong
Montreal / Doha
Manila
Brisbane
Dubai
Brussels
Dha
doha
Entebbe, Doha
Copenhagen
London
Adelaide
New York
Doha/Sydney
Delhi
Qatar
DOH
Sydney / Doha
Bangkok
Jakarta
Adelaide / Doha
Baku / Doha
Doha, Entebbe
BKK / DOH
DOA
BKK and DOH
DXB
BKK/DOH
BRU
Fukuoka
Tokyo Haneda
Haneda
Tokyo Narita
Chicago
NRT
HND
MUC
Dubai / Melbourne
Toronto
dubai
Dubai, Bangkok, Kuala Lumpur
Dubia
Harare
Athens
Melbourne and Dubai
Entebbe
Oslo
Seattle
Dhaka
Athens and Dubai
Lagoa
Heathrow
Bangkok / Dubai
Brisbane / Dubai
Dubai / Brisbane
New York JFK
Athens / Dubai
Duabi
Dubai / Bangkok
Dubai / Kuala Lumpur
Arlanda Sweden
Glasgow
DXB / JFK
A